In [15]:
import os
import math
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter
from tqdm import tqdm

# =====================
# CONFIG
# =====================

DATA_DIR = r"E:\ASL_Citizen\NEW\Top_Classes_Landmarks_Preprocessed_method2"

FEATURE_DIM = 438
MAX_LEN = 160
BATCH_SIZE = 16
EPOCHS = 60
LR = 1e-4
PATIENCE = 10

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

# =====================
# DATASET
# =====================

class LandmarkDataset(Dataset):
    def __init__(self, files, labels):
        self.files = files
        self.labels = labels

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        x = np.load(self.files[idx]).astype(np.float32)

        # Pad / Truncate
        if x.shape[0] > MAX_LEN:
            x = x[:MAX_LEN]
        else:
            pad = MAX_LEN - x.shape[0]
            x = np.pad(x, ((0, pad), (0, 0)))

        x = torch.tensor(x).permute(1, 0)  # (F, T)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

# =====================
# LOAD FILES
# =====================

files, labels = [], []
for f in os.listdir(DATA_DIR):
    if f.endswith(".npy"):
        files.append(os.path.join(DATA_DIR, f))
        labels.append(f.split("_")[0])

le = LabelEncoder()
y = le.fit_transform(labels)
NUM_CLASSES = len(le.classes_)

print("Samples:", len(files))
print("Classes:", NUM_CLASSES)

# =====================
# SPLIT
# =====================

X_train, X_tmp, y_train, y_tmp = train_test_split(
    files, y, test_size=0.2, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42
)

# =====================
# SAMPLER
# =====================

counter = Counter(y_train)
class_weights = {c: 1.0 / v for c, v in counter.items()}
sample_weights = [class_weights[y] for y in y_train]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

# =====================
# LOADERS
# =====================

train_ds = LandmarkDataset(X_train, y_train)
val_ds   = LandmarkDataset(X_val, y_val)
test_ds  = LandmarkDataset(X_test, y_test)

train_loader = DataLoader(train_ds, BATCH_SIZE, sampler=sampler)
val_loader   = DataLoader(val_ds, BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_ds, BATCH_SIZE, shuffle=False)

# =====================
# POSITIONAL ENCODING
# =====================

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=200):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

# =====================
# MODEL (SAFE IMPROVEMENT)
# =====================

class ASLModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()

        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 256)

        self.temporal = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )

        self.pos = PositionalEncoding(256)

        enc = nn.TransformerEncoderLayer(
            d_model=256,
            nhead=8,
            dim_feedforward=512,
            dropout=0.2,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(enc, num_layers=2)

        # 🔧 ONLY change: tiny dropout
        self.classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.15),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        # x: (B, F, T)
        x = x.permute(0, 2, 1)  # (B, T, F)

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))

        x = x.transpose(1, 2)
        x = self.temporal(x)
        x = x.transpose(1, 2)

        x = self.pos(x)
        x = self.transformer(x)

        # 🔧 Mean + Max pooling (SAFE)
        x_mean = x.mean(dim=1)
        x_max  = x.max(dim=1).values
        x = 0.7 * x_mean + 0.3 * x_max

        return self.classifier(x)

# =====================
# INIT
# =====================

model = ASLModel(FEATURE_DIM, NUM_CLASSES).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", factor=0.5, patience=3
)

print(model)

# =====================
# TRAIN
# =====================

def train():
    best = 0
    stop = 0

    for epoch in range(EPOCHS):
        model.train()
        correct, total = 0, 0

        for x, y in tqdm(train_loader):
            x, y = x.to(DEVICE), y.to(DEVICE)

            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

            pred = out.argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        train_acc = correct / total

        model.eval()
        vc, vt = 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                p = model(x).argmax(1)
                vc += (p == y).sum().item()
                vt += y.size(0)

        val_acc = vc / vt
        scheduler.step(val_acc)

        print(f"Epoch {epoch+1} | Train {train_acc:.4f} | Val {val_acc:.4f}")

        if val_acc > best:
            best = val_acc
            stop = 0
            torch.save(model.state_dict(), "best_model_safe.pth")
            print("Saved best model")
        else:
            stop += 1
            if stop >= PATIENCE:
                print("Early stopping")
                break

# =====================
# TEST
# =====================

def test():
    model.load_state_dict(torch.load("best_model_safe.pth"))
    model.eval()

    preds, gt = [], []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            p = model(x).argmax(1)
            preds.extend(p.cpu().numpy())
            gt.extend(y.cpu().numpy())

    acc = np.mean(np.array(preds) == np.array(gt))
    print("Test Accuracy:", acc)
    print(classification_report(gt, preds, target_names=le.classes_))
    print(confusion_matrix(gt, preds))

# =====================
# RUN
# =====================

train()
test()


Device: cpu
Samples: 5568
Classes: 146
ASLModel(
  (fc1): Linear(in_features=438, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (temporal): Sequential(
    (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (pos): PositionalEncoding()
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=512, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=512, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)

100%|████████████████████████████████████████████████████████████████████████████████| 279/279 [02:46<00:00,  1.68it/s]


Epoch 1 | Train 0.0216 | Val 0.0305
Saved best model


100%|████████████████████████████████████████████████████████████████████████████████| 279/279 [02:42<00:00,  1.72it/s]


Epoch 2 | Train 0.0694 | Val 0.0898
Saved best model


...

Epoch 58 | Train 0.9982 | Val 0.7971


100%|████████████████████████████████████████████████████████████████████████████████| 279/279 [02:24<00:00,  1.93it/s]


Epoch 59 | Train 0.9978 | Val 0.8061


100%|████████████████████████████████████████████████████████████████████████████████| 279/279 [02:20<00:00,  1.99it/s]


Epoch 60 | Train 0.9989 | Val 0.8079
Saved best model
Test Accuracy: 0.8061041292639138
              precision    recall  f1-score   support

       about       1.00      0.67      0.80         6
       after       0.50      0.33      0.40         3
       angry       1.00      1.00      1.00         3
       apple       0.60      1.00      0.75         3
        aunt       0.50      0.67      0.57         3
        baby       0.83      0.83      0.83         6
         bad       1.00      0.67      0.80         3
    bathroom       0.20      0.33      0.25         3
      before       0.75      1.00      0.86         3
         big       0.75      1.00      0.86         3
        bird       1.00      1.00      1.00         3
        blue       0.67      0.67      0.67         3
         boy       0.75      1.00      0.86         3
     brother       1.00      1.00      1.00         3
       brown       1.00      1.00      1.00         3
       brush       1.00      1.00      1.00    